# Enrichment sample in Python for Azure Cognitive Search

In this Jupyter Notebook, create and run enrichment steps to unlock searchable content in Azure blobs. It performs operations over mixed content in Azure Storage, such as images and application files, using a skillset that analyzes and extracts text information that becomes searchable in Azure Cognitive Search. The full documentation for this sample can be found at [Tutorial: Use Python and AI to generate searchable content from Azure blobs](https://docs.microsoft.com/azure/search/cognitive-search-tutorial-blob-python).

This sample creates the following objects on your search service:

+ search index
+ data source
+ skillset
+ indexer

In the last step, you'll run queries against the search index to explore the text output that was generated for each blob.

This notebook calls the [Search REST APIs](https://docs.microsoft.com/rest/api/searchservice/), but you can also use the Azure.Search.Documents client library in the Azure SDK for Python to perform the same steps. See this [Python quickstart](https://docs.microsoft.com/azure/search/search-get-started-python) for details.

To run this sample, satisfy all [prerequisites](https://docs.microsoft.com/azure/search/cognitive-search-tutorial-blob-python#prerequisites) stated in the tutorial and upload the sample data to a blob container in Azure Storage account. In this notebook, replace the placeholders for the search service endpoint, the admin API key, Azure Storage connection string, and blob container. Once you've provided all four values, you can run all cells, but the query won't return results until the indexer is finished and the search index is loaded. 

We recommend running each step and making sure it completes before moving on.

In [1]:
import json
import requests
from pprint import pprint

In [2]:
# Define the names for the data source, skillset, index and indexer
datasource_name = "cogsrch-py-datasource-new"
skillset_name = "cogsrch-py-skillset-new"
index_name = "cogsrch-py-index-new"
indexer_name = "cogsrch-py-indexer-new"

In [3]:
# Setup the endpoint
endpoint = 'https://mydemo.search.windows.net'
headers = {'Content-Type': 'application/json',
           'api-key': 'oBjkMG4ew9dsNVNj6gGj9TorpbDs22L9GWQj7DHTvHAzSeCR8mDe'}
params = {
    'api-version': '2020-06-30'
}

In [4]:
# Create a data source
# This data source points to your Azure Storage account.
# You should already have a blob container that contains the sample data
datasourceConnectionString = "DefaultEndpointsProtocol=https;AccountName=sdai;AccountKey=W7/2Zf/JkrZSH3mKBQFmLXzIcsMtP9ZVrYOaqmq9/fKQOikhmLUhZ+99rBTJnCugC6hTKPoJiNKG+AStEsUgzw==;EndpointSuffix=core.windows.net"
datasource_payload = {
    "name": datasource_name,
    "description": "Demo files to demonstrate cognitive search capabilities.",
    "type": "azureblob",
    "credentials": {
        "connectionString": datasourceConnectionString
    },
    "container": {
        "name": "new-blob-service"
    }
}
r = requests.put(endpoint + "/datasources/" + datasource_name,
                 data=json.dumps(datasource_payload), headers=headers, params=params)
print(r.status_code)

201


In [5]:
# Create a skillset
skillset_payload = {
    "name": skillset_name,
    "description":
    "Extract entities, detect language and extract key-phrases",
    "skills":
    [
        {
            "@odata.type": "#Microsoft.Skills.Text.V3.EntityRecognitionSkill",
            "categories": ["Organization"],
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text", 
                    "source": "/document/content"
                }
            ],
            "outputs": [
                {
                    "name": "organizations", 
                    "targetName": "organizations"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.LanguageDetectionSkill",
            "inputs": [
                {
                    "name": "text", 
                    "source": "/document/content"
                }
            ],
            "outputs": [
                {
                    "name": "languageCode",
                    "targetName": "languageCode"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
            "textSplitMode": "pages",
            "maximumPageLength": 4000,
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/content"
                },
                {
                    "name": "languageCode",
                    "source": "/document/languageCode"
                }
            ],
            "outputs": [
                {
                    "name": "textItems",
                    "targetName": "pages"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.KeyPhraseExtractionSkill",
            "context": "/document/pages/*",
            "inputs": [
                {
                    "name": "text", 
                    "source": "/document/pages/*"
                },
                {
                    "name": "languageCode", 
                    "source": "/document/languageCode"
                }
            ],
            "outputs": [
                {
                    "name": "keyPhrases",
                    "targetName": "keyPhrases"
                }
            ]
        }
    ]
}

r = requests.put(endpoint + "/skillsets/" + skillset_name,
                 data=json.dumps(skillset_payload), headers=headers, params=params)
print(r.status_code)

201


In [6]:
# Create an index
# Queries operate over the searchable fields and filterable fields in the index
index_payload = {
    "name": index_name,
    "fields": [
        {
            "name": "id",
            "type": "Edm.String",
            "key": "true",
            "searchable": "true",
            "filterable": "false",
            "facetable": "false",
            "sortable": "true"
        },
        {
            "name": "content",
            "type": "Edm.String",
            "sortable": "false",
            "searchable": "true",
            "filterable": "false",
            "facetable": "false"
        },
        {
            "name": "languageCode",
            "type": "Edm.String",
            "searchable": "true",
            "filterable": "false",
            "facetable": "false"
        },
        {
            "name": "keyPhrases",
            "type": "Collection(Edm.String)",
            "searchable": "true",
            "filterable": "false",
            "facetable": "false"
        },
        {
            "name": "organizations",
            "type": "Collection(Edm.String)",
            "searchable": "true",
            "sortable": "false",
            "filterable": "false",
            "facetable": "false"
        }
    ]
}

r = requests.put(endpoint + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)

201


In [7]:
# Create an indexer
indexer_payload = {
    "name": indexer_name,
    "dataSourceName": datasource_name,
    "targetIndexName": index_name,
    "skillsetName": skillset_name,
    "fieldMappings": [
        {
            "sourceFieldName": "metadata_storage_path",
            "targetFieldName": "id",
            "mappingFunction":
            {"name": "base64Encode"}
        },
        {
            "sourceFieldName": "content",
            "targetFieldName": "content"
        }
    ],
    "outputFieldMappings":
    [
        {
            "sourceFieldName": "/document/organizations",
            "targetFieldName": "organizations"
        },
        {
            "sourceFieldName": "/document/pages/*/keyPhrases/*",
            "targetFieldName": "keyPhrases"
        },
        {
            "sourceFieldName": "/document/languageCode",
            "targetFieldName": "languageCode"
        }
    ],
    "parameters":
    {
        "maxFailedItems": -1,
        "maxFailedItemsPerBatch": -1,
        "configuration":
        {
            "dataToExtract": "contentAndMetadata",
            "imageAction": "generateNormalizedImages"
        }
    }
}

r = requests.put(endpoint + "/indexers/" + indexer_name,
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r.status_code)

201


In [8]:
# Optionally, get indexer status to confirm that it's running
r = requests.get(endpoint + "/indexers/" + indexer_name +
                 "/status", headers=headers, params=params)
pprint(json.dumps(r.json(), indent=1))

('{\n'
 ' "@odata.context": '
 '"https://mydemo.search.windows.net/$metadata#Microsoft.Azure.Search.V2020_06_30.IndexerExecutionInfo",\n'
 ' "name": "cogsrch-py-indexer-new",\n'
 ' "status": "running",\n'
 ' "lastResult": {\n'
 '  "status": "inProgress",\n'
 '  "errorMessage": null,\n'
 '  "startTime": "2022-09-19T02:31:21.75Z",\n'
 '  "endTime": null,\n'
 '  "itemsProcessed": 0,\n'
 '  "itemsFailed": 0,\n'
 '  "initialTrackingState": null,\n'
 '  "finalTrackingState": null,\n'
 '  "errors": [],\n'
 '  "warnings": [],\n'
 '  "metrics": null\n'
 ' },\n'
 ' "executionHistory": [],\n'
 ' "limits": {\n'
 '  "maxRunTime": "PT2H",\n'
 '  "maxDocumentExtractionSize": 134217728,\n'
 '  "maxDocumentContentCharactersToExtract": 4194304\n'
 ' }\n'
 '}')


In [9]:
# Query the service for the index definition
# Query responses can be verbose. If you get "Output exceeds the size limit. Open the full output data in a text editor", open the output in an editor.
r = requests.get(endpoint + "/indexes/" + index_name,
                 headers=headers, params=params)
pprint(json.dumps(r.json(), indent=1))

('{\n'
 ' "@odata.context": '
 '"https://mydemo.search.windows.net/$metadata#indexes/$entity",\n'
 ' "@odata.etag": "\\"0x8DA99E6F8CB1757\\"",\n'
 ' "name": "cogsrch-py-index-new",\n'
 ' "defaultScoringProfile": null,\n'
 ' "fields": [\n'
 '  {\n'
 '   "name": "id",\n'
 '   "type": "Edm.String",\n'
 '   "searchable": true,\n'
 '   "filterable": false,\n'
 '   "retrievable": true,\n'
 '   "sortable": true,\n'
 '   "facetable": false,\n'
 '   "key": true,\n'
 '   "indexAnalyzer": null,\n'
 '   "searchAnalyzer": null,\n'
 '   "analyzer": null,\n'
 '   "synonymMaps": []\n'
 '  },\n'
 '  {\n'
 '   "name": "content",\n'
 '   "type": "Edm.String",\n'
 '   "searchable": true,\n'
 '   "filterable": false,\n'
 '   "retrievable": true,\n'
 '   "sortable": false,\n'
 '   "facetable": false,\n'
 '   "key": false,\n'
 '   "indexAnalyzer": null,\n'
 '   "searchAnalyzer": null,\n'
 '   "analyzer": null,\n'
 '   "synonymMaps": []\n'
 '  },\n'
 '  {\n'
 '   "name": "languageCode",\n'
 '   "type": "Edm.S

In [10]:
# Query the index to return the contents of "organizations", created through Entity Recognition during enrichment
# For keyword search, replace the asterisk with comma-separated query terms: search=microsoft,azure
r = requests.get(endpoint + "/indexes/" + index_name +
                 "/docs?&search=*&$select=organizations", headers=headers, params=params)
pprint(json.dumps(r.json(), indent=1))

('{\n'
 ' "@odata.context": '
 '"https://mydemo.search.windows.net/indexes(\'cogsrch-py-index-new\')/$metadata#docs(*)",\n'
 ' "value": []\n'
 '}')


This step concludes the walkthrough, showing you how to invoke skills that extract searchable content and information from image and application files in Azure Storage. The processes run on Azure Cognitive Search, and the resulting index can be used in client apps to provide a search experience. For more information, visit the [official docs](https://docs.microsoft.com/azure/search/).